In [43]:
#Libraries
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plts
from datetime import date
import pandas_datareader as data
from pandas_datareader import *
import math
from PIL import Image
import yfinance as yf
import pickle

#Scaler
from sklearn.metrics import classification_report
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler 
from sklearn.metrics import mean_squared_error
from numpy import sqrt 


#Model import
from sklearn.model_selection import train_test_split
from keras.models import load_model
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from statsmodels.tsa.arima.model import ARIMA
from pmdarima.arima import auto_arima
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.arima_model import ARIMAResults
from sklearn import tree 

#Model Result Analysis
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, ConfusionMatrixDisplay, precision_score, recall_score, f1_score, classification_report, roc_curve, plot_roc_curve, auc, precision_recall_curve, plot_precision_recall_curve, average_precision_score
from sklearn.model_selection import cross_val_score

#Streamlit
import streamlit as st
from plotly import graph_objs as plt

In [44]:
#Default-Setting Dates for scarping Dataset
START = '2019-01-01'
END = date.today().strftime("%Y-%m-%d")

stock_dataset = data.DataReader('AAPL', 'yahoo', START, END)

In [45]:
#Stochastic Oscillator
stock_dataset['14-high'] = stock_dataset['High'].rolling(14).max()
stock_dataset['14-low'] = stock_dataset['Low'].rolling(14).min()
stock_dataset['%K'] = (stock_dataset['Close'] - stock_dataset['14-low'])*100/(stock_dataset['14-high'] - stock_dataset['14-low'])
stock_dataset['%D'] = stock_dataset['%K'].rolling(3).mean()

stock_dataset.drop(['14-high', '14-low'], axis = 1, inplace=True)

# Adding Buy/Sell Signals from Sotchastic Oscillator Indicator
def categorise_so(row):  
    if row['%K'] <= 20 and row['%D'] <= 20:
        return 'Buy'
    elif row['%K'] >= 80 and  row['%D']>= 80:
        return 'Sell'
    else:
        return 'Hold'
   
stock_dataset['SO Indicator'] = stock_dataset.apply(lambda row: categorise_so(row), axis=1)

In [46]:
#RSI
delta = stock_dataset['Close'].diff()
up = delta.clip(lower=0)
down = -1*delta.clip(upper=0)
ema_up = up.ewm(com=13, adjust=False).mean()
ema_down = down.ewm(com=13, adjust=False).mean()
rs = ema_up/ema_down
stock_dataset['RSI'] = 100 - (100/(1 + rs))

# Adding Buy/Sell Signals from RSI Indicator
def categorise_rsi(row):  
    if row['RSI'] <= 30:
        return 'Buy'
    elif row['RSI'] >= 70:
        return 'Sell'
    else:
        return 'Hold'
  
stock_dataset['RSI Indicator'] = stock_dataset.apply(lambda row: categorise_rsi(row), axis=1)

In [47]:
#Bollinger Bands
def get_sma(prices, rate):
    return prices.rolling(rate).mean()

def get_bollinger_bands(prices, rate=20):
    # SMA for 20 Days (Middle Band)
    sma = get_sma(prices, rate)
    std = prices.rolling(rate).std()

    # Calculating Upper Band
    bollinger_upper = sma + (std * 2 )

    # Calculate Lower Band
    bollinger_lower = sma - (std * 2 )

    #Middle Band
    bollinger_middle = sma
    return bollinger_upper, bollinger_lower, bollinger_middle


closing_prices = stock_dataset['Close']
bollinger_upper, bollinger_lower, bollinger_middle = get_bollinger_bands(closing_prices)

#Adding Bollinger Bands to the Dataset
stock_dataset['Bollinger_Upper'] = bollinger_upper
stock_dataset['Bollinger_Lower'] = bollinger_lower

# Adding Buy/Sell Signals from Bollinger Bands Indicator
def categorise_bollinger(row):  
    if row['Close'] < row['Bollinger_Lower']:
        return 'Buy'
    elif row['Close'] > row['Bollinger_Upper']:
        return 'Sell'
    else:
        return 'Hold'
    
   
stock_dataset['Bollinger Indicator'] = stock_dataset.apply(lambda row: categorise_bollinger(row), axis=1)

In [48]:
#MACD Inidicator
# Calculating the MACD Line and the Signal Line
ema12 = stock_dataset['Close'].ewm(span=12, adjust=False).mean()
ema26 = stock_dataset['Close'].ewm(span=26, adjust=False).mean()
macd = ema12 - ema26
signal = macd.ewm(span=9, adjust=False).mean()


#Appeding the MACD and Signal Data to Dataset
stock_dataset['MACD'] = macd
stock_dataset['Signal'] = signal

#Adding Target Variable
# Recommender Based on the three indicators
def recommender(row): 
    # If All Share the Same Signal OutPut That Signal (Buy)
    if row['RSI Indicator'] == "Buy" and row['SO Indicator'] == 'Buy' and  row['Bollinger Indicator'] == 'Buy':
        return 'Buy'
     # If Any 2 Indicators Share the Same Signal Output That Signal (buy)
    elif row['RSI Indicator'] == 'Buy' and row['SO Indicator'] == 'Buy':
        return 'Buy'
    elif row['RSI Indicator'] == 'Buy' and row['Bollinger Indicator'] == 'Buy':
        return 'Buy'
    elif row['SO Indicator'] == 'Buy' and row['Bollinger Indicator'] == 'Buy':
        return 'Buy'   
    # If All Share the Same Signal Output That Signal (Sell)
    elif row['RSI Indicator'] == 'Sell' and row['SO Indicator'] == 'Sell' and  row['Bollinger Indicator'] == 'Sell':
        return 'Sell'
    # If Any 2 Indicators Share the Same Signal Output That Signal (Sell)
    elif row['RSI Indicator'] == 'Sell' and row['SO Indicator'] == 'Sell':
        return 'Sell'
    elif row['RSI Indicator'] == 'Sell' and row['Bollinger Indicator'] == 'Sell':
        return 'Sell'
    elif row['SO Indicator'] == 'Sell' and row['Bollinger Indicator'] == 'Sell':
        return 'Sell'    
    # If All Share the Same Signal OutPut That Signal (Hold)
    elif row['RSI Indicator'] == 'Hold' and row['SO Indicator'] == 'Hold' and  row['Bollinger Indicator'] == 'Hold':
        return 'Hold'
    # If Any 2 Indicators Share the Same Signal Output That Signal (Hold)
    elif row['RSI Indicator'] == 'Hold' and row['SO Indicator'] == 'Hold':
        return 'Hold'
    elif row['RSI Indicator'] == 'Hold' and row['Bollinger Indicator'] == 'Hold':
        return 'Hold'
    elif row['SO Indicator'] == 'Hold' and row['Bollinger Indicator'] == 'Hold':
        return 'Hold'      
    else:
        return 'Unclassed'

stock_dataset['Recommender'] = stock_dataset.apply(lambda row: recommender(row), axis=1)

In [51]:

#Dropping Other Trading Signals from Indicators
stock_dataset.drop(['RSI Indicator', 'SO Indicator', 'Bollinger Indicator', 'Adj Close'], axis = 1, inplace=True)

#Dropping Null Values
stock_dataset = stock_dataset.dropna()

#Encoding Categorical Variables

signal_dict = {'Hold': 0, 'Sell': 1, 'Buy': 2}
stock_dataset['Recommender'] = stock_dataset['Recommender'].map(signal_dict)

C:\Users\RAHUL\AppData\Local\Temp/ipykernel_18832/3435696030.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  stock_dataset['Recommender'] = stock_dataset['Recommender'].map(signal_dict)


In [52]:
#Dropping Null Values
stock_dataset = stock_dataset.dropna()

In [53]:
stock_dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 822 entries, 2019-01-31 to 2022-05-04
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   High             822 non-null    float64
 1   Low              822 non-null    float64
 2   Open             822 non-null    float64
 3   Close            822 non-null    float64
 4   Volume           822 non-null    float64
 5   %K               822 non-null    float64
 6   %D               822 non-null    float64
 7   RSI              822 non-null    float64
 8   Bollinger_Upper  822 non-null    float64
 9   Bollinger_Lower  822 non-null    float64
 10  MACD             822 non-null    float64
 11  Signal           822 non-null    float64
 12  Recommender      822 non-null    float64
dtypes: float64(13)
memory usage: 89.9 KB


In [54]:
# converting 'Weight' from float to int
stock_dataset['Recommender'] = stock_dataset['Recommender'].astype(int)

In [55]:
stock_dataset.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 822 entries, 2019-01-31 to 2022-05-04
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   High             822 non-null    float64
 1   Low              822 non-null    float64
 2   Open             822 non-null    float64
 3   Close            822 non-null    float64
 4   Volume           822 non-null    float64
 5   %K               822 non-null    float64
 6   %D               822 non-null    float64
 7   RSI              822 non-null    float64
 8   Bollinger_Upper  822 non-null    float64
 9   Bollinger_Lower  822 non-null    float64
 10  MACD             822 non-null    float64
 11  Signal           822 non-null    float64
 12  Recommender      822 non-null    int32  
dtypes: float64(12), int32(1)
memory usage: 86.7 KB


In [58]:
#Independant and Dependent Variable
y=stock_dataset.iloc[:,12:13] #Dependent variable
X=stock_dataset.iloc[:,0:12] #Independent variable

In [59]:
#Spliting the data into train and test
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.3, random_state=42)

#Scaling
scaler = MinMaxScaler(feature_range=(0,1))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


In [62]:
rf_tuned_model = 'rf_tuned_model.sav'
loaded_model = pickle.load(open(rf_tuned_model, 'rb'))

#Making Prediction with Model
y_pred = loaded_model.predict(X_test)

#Evaluate
accuracy = accuracy_score(y_pred , y_test)
print("Accuracy Score :", accuracy)

#Precision
test_precision = precision_score(y_test, y_pred,  average='macro')
print("Precision Score :", test_precision) 
#Recall
test_recall = recall_score(y_test, y_pred,  average='macro')
print("Recall Score : ", test_recall)
#F1 Score
test_f1 = f1_score(y_test, y_pred,average='macro')
print("F1 Score: ", test_f1)

Accuracy Score : 0.8906882591093117
Precision Score : 0.6679556475571656
Recall Score :  0.9222085385878489
F1 Score:  0.7149589913086057
